In [ ]:
import math, pickle, pandas as pd, numpy as np
from moire_model import L, hbar2_over_2m, phi, V0, a_m, cut, n_sup, N_e
from moire_model import energy_static, moire_potential, coulomb_ewald_2D, b_vectors
from plane_wave_moire import build_Hk, build_G_basis
from make_bloch_table import build_bloch_table #build_bloch_table(a_m, V0, phi, hbar2_over_2m, n_sup, N_e, cut)
from k_selection import occupied_k_points
# from math import erfc, sqrt, pi, exp

#### benchmark for 3 electrons from scratch (consistent)

In [2]:
bloch_table = build_bloch_table(
    a_m, V0, phi, hbar2_over_2m,
    n_sup, N_e, cut
)
# save for later comparison
with open("bloch_table_3electrons.pkl","wb") as f:
    pickle.dump(bloch_table, f)
print(f"Done: built Bloch table with {len(bloch_table)} entries.")

g list [array([-0.45169951,  0.78236649]), array([-0.45169951, -0.78236649]), array([ 9.03399011e-01, -2.21268941e-16])]
occuplied k [[-0.          0.        ]
 [-0.1505665  -0.26078883]
 [-0.1505665   0.26078883]]
g list [array([-0.45169951,  0.78236649]), array([-0.45169951, -0.78236649]), array([ 9.03399011e-01, -2.21268941e-16])]
G list [(-2, -2, array([ 1.80679802e+00, -8.88178420e-16])), (-2, -1, array([ 1.35509852, -0.78236649])), (-2, 0, array([ 0.90339901, -1.56473299])), (-2, 1, array([ 0.45169951, -2.34709948])), (-2, 2, array([-1.22124533e-15, -3.12946597e+00])), (-1, -2, array([1.35509852, 0.78236649])), (-1, -1, array([ 9.03399011e-01, -4.44089210e-16])), (-1, 0, array([ 0.45169951, -0.78236649])), (-1, 1, array([-6.10622664e-16, -1.56473299e+00])), (-1, 2, array([-0.45169951, -2.34709948])), (0, -2, array([0.90339901, 1.56473299])), (0, -1, array([0.45169951, 0.78236649])), (0, 0, array([-0.,  0.])), (0, 1, array([-0.45169951, -0.78236649])), (0, 2, array([-0.90339901, -

In [5]:
with open("bloch_table_3electrons.pkl", "rb") as fh:
    bloch = pickle.load(fh)          # list of 6 dictionaries
rows = []
for idx, entry in enumerate(bloch):
    kx, ky   = entry["k"]            # Bloch momentum components (nm⁻¹)
    eps      = entry["eps"]          # lowest-band eigenvalue (meV)
    n_G      = len(entry["coeff"])   # number of plane waves (should be 25)
    rows.append({"k_x (nm⁻¹)": kx,
                 "k_y (nm⁻¹)": ky,
                 "ε₀  (meV)"  : eps,
                 "# G-vectors": n_G})
df = pd.DataFrame(rows)
print("sum:", df['ε₀  (meV)'].sum())
df

sum: -21.402247597875277


,k_x (nm⁻¹),k_y (nm⁻¹),ε₀ (meV),# G-vectors
0,-0.000000,0.000000,-11.988679,25
1,-0.150567,-0.260789,-4.706784,25
2,-0.150567,0.260789,-4.706784,25


In [6]:
# Build table again
G_list, idx_map = build_G_basis(cut, a_m)
k_points = occupied_k_points(N_e, n_sup, a_m)
bloch_table = []
Hc_list = []
for k in k_points:
    Hk = build_Hk(k, G_list, idx_map, V0, phi, hbar2_over_2m)
    evals, evecs = np.linalg.eigh(Hk)
    c0 = evecs[:,0]
    Hc_list.append(Hk.dot(c0))
    bloch_table.append({"coeff":c0, "kG":np.array([k+G for *_,G in G_list])})

g list [array([-0.45169951,  0.78236649]), array([-0.45169951, -0.78236649]), array([ 9.03399011e-01, -2.21268941e-16])]
G list [(-2, -2, array([ 1.80679802e+00, -8.88178420e-16])), (-2, -1, array([ 1.35509852, -0.78236649])), (-2, 0, array([ 0.90339901, -1.56473299])), (-2, 1, array([ 0.45169951, -2.34709948])), (-2, 2, array([-1.22124533e-15, -3.12946597e+00])), (-1, -2, array([1.35509852, 0.78236649])), (-1, -1, array([ 9.03399011e-01, -4.44089210e-16])), (-1, 0, array([ 0.45169951, -0.78236649])), (-1, 1, array([-6.10622664e-16, -1.56473299e+00])), (-1, 2, array([-0.45169951, -2.34709948])), (0, -2, array([0.90339901, 1.56473299])), (0, -1, array([0.45169951, 0.78236649])), (0, 0, array([-0.,  0.])), (0, 1, array([-0.45169951, -0.78236649])), (0, 2, array([-0.90339901, -1.56473299])), (1, -2, array([0.45169951, 2.34709948])), (1, -1, array([6.10622664e-16, 1.56473299e+00])), (1, 0, array([-0.45169951,  0.78236649])), (1, 1, array([-9.03399011e-01,  4.44089210e-16])), (1, 2, array([

In [7]:
# ─── 2) Local energy via Fourier method ────────────────────────────────────────
def slater_matrices(R):
    M = np.zeros((N_e,N_e), dtype=complex)
    Mprime = np.zeros((N_e,N_e), dtype=complex)
    for i, r in enumerate(R):
        for j, orb in enumerate(bloch_table):
            phase = np.exp(1j * (orb["kG"] @ r))
            M[i,j]      = np.dot(orb["coeff"], phase)
            Mprime[i,j] = np.dot(Hc_list[j], phase)
    return M, Mprime

def local_energy_fourier(R):
    M, Mprime = slater_matrices(R)
    invM = np.linalg.inv(M)
    return sum(invM[j,i] * Mprime[i,j] for i in range(N_e) for j in range(N_e)).real

# ─── 3) Quantum Monte Carlo sampling ───────────────────────────────────────────
rng = np.random.default_rng(123)
step_size = 0.5
n_steps   = 10000
warmup    = 2000
L = n_sup * a_m

# initialize random positions
R = rng.random((N_e,2)) * L
psi_current = np.linalg.det(slater_matrices(R)[0])

E_samples = []
for step in range(n_steps):
    i = rng.integers(N_e)
    r_old = R[i].copy()
    R[i] = (r_old + step_size*(rng.random(2)-0.5)) % L
    M_new = slater_matrices(R)[0]
    psi_new = np.linalg.det(M_new)
    if rng.random() < abs(psi_new/psi_current)**2:
        psi_current = psi_new
    else:
        R[i] = r_old
    if step >= warmup:
        E_samples.append(local_energy_fourier(R))
E_avg = np.mean(E_samples)
E_std = np.std(E_samples)
print(f"QMC average ⟨E⟩: {E_avg:.15f} ± {E_std:.15f} meV")

QMC average ⟨E⟩: -21.402247597875416 ± 0.000000000000466 meV


#### benchmark for 6 electrons from scratch (consistent) + interaction energy

In [2]:
bloch_table = build_bloch_table(
    a_m, V0, phi, hbar2_over_2m,
    n_sup, N_e, cut
)
# save for later comparison
with open("bloch_table_6electrons.pkl","wb") as f:
    pickle.dump(bloch_table, f)
print(f"Done: built Bloch table with {len(bloch_table)} entries.")

g list [array([-0.45169951,  0.78236649]), array([-0.45169951, -0.78236649]), array([ 9.03399011e-01, -2.21268941e-16])]
occuplied k [[-0.00000000e+00  0.00000000e+00]
 [-1.50566502e-01 -2.60788831e-01]
 [-1.50566502e-01  2.60788831e-01]
 [-3.01133004e-01  1.11022302e-16]
 [-4.51699505e-01 -2.60788831e-01]
 [-4.51699505e-01  2.60788831e-01]]
g list [array([-0.45169951,  0.78236649]), array([-0.45169951, -0.78236649]), array([ 9.03399011e-01, -2.21268941e-16])]
G list [(-2, -2, array([ 1.80679802e+00, -8.88178420e-16])), (-2, -1, array([ 1.35509852, -0.78236649])), (-2, 0, array([ 0.90339901, -1.56473299])), (-2, 1, array([ 0.45169951, -2.34709948])), (-2, 2, array([-1.22124533e-15, -3.12946597e+00])), (-1, -2, array([1.35509852, 0.78236649])), (-1, -1, array([ 9.03399011e-01, -4.44089210e-16])), (-1, 0, array([ 0.45169951, -0.78236649])), (-1, 1, array([-6.10622664e-16, -1.56473299e+00])), (-1, 2, array([-0.45169951, -2.34709948])), (0, -2, array([0.90339901, 1.56473299])), (0, -1, arr

In [3]:
with open("bloch_table_6electrons.pkl", "rb") as fh:
    bloch = pickle.load(fh)          # list of 6 dictionaries
rows = []
for idx, entry in enumerate(bloch):
    kx, ky   = entry["k"]            # Bloch momentum components (nm⁻¹)
    eps      = entry["eps"]          # lowest-band eigenvalue (meV)
    n_G      = len(entry["coeff"])   # number of plane waves (should be 25)
    rows.append({"k_x (nm⁻¹)": kx,
                 "k_y (nm⁻¹)": ky,
                 "ε₀  (meV)"  : eps,
                 "# G-vectors": n_G})
df = pd.DataFrame(rows)
print("sum:", df['ε₀  (meV)'].sum())
df

sum: -21.582481657883854


,k_x (nm⁻¹),k_y (nm⁻¹),ε₀ (meV),# G-vectors
0,-0.000000,0.000000e+00,-11.988679,25
1,-0.150567,-2.607888e-01,-4.706784,25
2,-0.150567,2.607888e-01,-4.706784,25
3,-0.301133,1.110223e-16,-4.705304,25
4,-0.451700,-2.607888e-01,2.262535,25
5,-0.451700,2.607888e-01,2.262535,25


In [4]:
# check bloch coefficients
idx   = 0                    # pick the Γ-point
c_G   = bloch[idx]["coeff"]   # complex length-25 vector
kplus = bloch[idx]["kG"]      # 25 corresponding k+G vectors
print("\ncoeffs:", c_G)  # should be (25,)

print("\nnormalisation ⟨ψ|ψ⟩:", np.sum(np.abs(c_G)**2))  # should be 1.0
print("energy sum", df["ε₀  (meV)"].sum())
print('energy average', df["ε₀  (meV)"].sum()/3 )


coeffs: [ 4.88042488e-03-0.00000000e+00j  5.94635876e-03+1.20259609e-02j
 -2.97522097e-03+3.88907796e-03j -7.73873874e-05+4.13746874e-04j
  5.14057733e-06+1.03947763e-05j  5.94635876e-03+1.20259609e-02j
  1.08567479e-01+7.26228402e-02j -8.18899182e-03+1.30370433e-01j
  5.96701190e-03+1.20659121e-02j  3.75782461e-04+1.89638671e-04j
 -2.97522097e-03+3.88907796e-03j -8.18899182e-03+1.30370433e-01j
  4.19699392e-01+8.48675359e-01j  1.08577457e-01+7.26253747e-02j
  4.89661645e-03-3.81859080e-06j -7.73873874e-05+4.13746874e-04j
  5.96701190e-03+1.20659121e-02j  1.08577457e-01+7.26253747e-02j
 -8.18494962e-03+1.30360965e-01j  5.94779841e-03+1.20252489e-02j
  5.14057733e-06+1.03947763e-05j  3.75782461e-04+1.89638671e-04j
  4.89661645e-03-3.81859080e-06j  5.94779841e-03+1.20252489e-02j
 -2.96235821e-03+3.87852818e-03j]

normalisation ⟨ψ|ψ⟩: 0.9999999999999969
energy sum -21.582481657883854
energy average -7.1941605526279515


In [6]:
# Build table again
G_list, idx_map = build_G_basis(cut, a_m)
k_points = occupied_k_points(N_e, n_sup, a_m)
bloch_table = []
Hc_list = []
for k in k_points:
    Hk = build_Hk(k, G_list, idx_map, V0, phi, hbar2_over_2m)
    evals, evecs = np.linalg.eigh(Hk)
    c0 = evecs[:,0]
    Hc_list.append(Hk.dot(c0))
    bloch_table.append({"coeff":c0, "kG":np.array([k+G for *_,G in G_list])})

g list [array([-0.45169951,  0.78236649]), array([-0.45169951, -0.78236649]), array([ 9.03399011e-01, -2.21268941e-16])]
G list [(-2, -2, array([ 1.80679802e+00, -8.88178420e-16])), (-2, -1, array([ 1.35509852, -0.78236649])), (-2, 0, array([ 0.90339901, -1.56473299])), (-2, 1, array([ 0.45169951, -2.34709948])), (-2, 2, array([-1.22124533e-15, -3.12946597e+00])), (-1, -2, array([1.35509852, 0.78236649])), (-1, -1, array([ 9.03399011e-01, -4.44089210e-16])), (-1, 0, array([ 0.45169951, -0.78236649])), (-1, 1, array([-6.10622664e-16, -1.56473299e+00])), (-1, 2, array([-0.45169951, -2.34709948])), (0, -2, array([0.90339901, 1.56473299])), (0, -1, array([0.45169951, 0.78236649])), (0, 0, array([-0.,  0.])), (0, 1, array([-0.45169951, -0.78236649])), (0, 2, array([-0.90339901, -1.56473299])), (1, -2, array([0.45169951, 2.34709948])), (1, -1, array([6.10622664e-16, 1.56473299e+00])), (1, 0, array([-0.45169951,  0.78236649])), (1, 1, array([-9.03399011e-01,  4.44089210e-16])), (1, 2, array([

In [7]:
# ─── 2) Local energy via Fourier method ────────────────────────────────────────
def slater_matrices(R):
    M = np.zeros((N_e,N_e), dtype=complex)
    Mprime = np.zeros((N_e,N_e), dtype=complex)
    for i, r in enumerate(R):
        for j, orb in enumerate(bloch_table):
            phase = np.exp(1j * (orb["kG"] @ r))
            M[i,j]      = np.dot(orb["coeff"], phase)
            Mprime[i,j] = np.dot(Hc_list[j], phase)
    return M, Mprime

def local_energy_fourier(R):
    M, Mprime = slater_matrices(R)
    invM = np.linalg.inv(M)
    return sum(invM[j,i] * Mprime[i,j] for i in range(N_e) for j in range(N_e)).real

# ─── 3) Quantum Monte Carlo sampling ───────────────────────────────────────────
rng = np.random.default_rng(123)
step_size = 0.5
n_steps   = 10000
warmup    = 2000
L = n_sup * a_m

# initialize random positions
R = rng.random((N_e,2)) * L
psi_current = np.linalg.det(slater_matrices(R)[0])

E_samples = []
for step in range(n_steps):
    i = rng.integers(N_e)
    r_old = R[i].copy()
    R[i] = (r_old + step_size*(rng.random(2)-0.5)) % L
    M_new = slater_matrices(R)[0]
    psi_new = np.linalg.det(M_new)
    if rng.random() < abs(psi_new/psi_current)**2:
        psi_current = psi_new
    else:
        R[i] = r_old
    if step >= warmup:
        E_samples.append(local_energy_fourier(R))
E_avg = np.mean(E_samples)
E_std = np.std(E_samples)
print(f"QMC average ⟨E⟩: {E_avg:.15f} ± {E_std:.15f} meV")

QMC average ⟨E⟩: -21.582481657883925 ± 0.000000000000435 meV


Check E_expect that includes interaction energy:

In [8]:
def local_energy_total(R):
    """One-body part   +   two-body Coulomb part (Ewald)."""
    return local_energy_fourier(R) + coulomb_ewald_2D(R)


# again, Quantum Monte Carlo sampling, but now use local_energy_total instead ───────────────────────────────────────────
rng = np.random.default_rng(123)
step_size = 0.5
n_steps   = 10000
warmup    = 2000
L = n_sup * a_m

# initialize random positions
R = rng.random((N_e,2)) * L
psi_current = np.linalg.det(slater_matrices(R)[0])

E_samples = []
for step in range(n_steps):
    i = rng.integers(N_e)
    r_old = R[i].copy()
    R[i] = (r_old + step_size*(rng.random(2)-0.5)) % L
    M_new = slater_matrices(R)[0]
    psi_new = np.linalg.det(M_new)
    if rng.random() < abs(psi_new/psi_current)**2:
        psi_current = psi_new
    else:
        R[i] = r_old
    if step >= warmup:
        E_samples.append(local_energy_total(R))
E_avg = np.mean(E_samples)
E_std = np.std(E_samples)
print("kinetic + moiré + electron–electron energy per Monte-Carlo configuration:")
print(f"QMC average ⟨E⟩ : {E_avg:.15f} ± {E_std:.15f} meV")

kinetic + moiré + electron–electron energy per Monte-Carlo configuration:
QMC average ⟨E⟩ : -16.324655746502970 ± 1.096285493643891 meV


In [10]:
# # 1) Non-interacting band sum
# E_band = df["ε₀  (meV)"].sum()
# print("Non-interacting sum:", E_band, "meV")

# # 2) Average one-body part during the MC run
# E1_samples = []
# for step in range(warmup, n_steps):
#     E1_samples.append(local_energy_fourier(R_snapshots[step]))  # store R during the loop if you need
# E1_avg = np.mean(E1_samples)

# print("⟨one-body⟩:", E1_avg, "meV")
# print("⟨Coulomb⟩   (= total – one-body):", E_avg - E1_avg, "meV")


#### check single electron E expectation:

In [49]:
# ─── Compute expectation via operator ────────────────────────────────────────
rows = []
# nonzero Fourier modes of the moiré potential
Gs = b_vectors(a_m)
Vq = {}
for gj in Gs:
    Vq[tuple(gj)]  = -V0 * np.exp( 1j*phi)
    Vq[tuple(-gj)] = -V0 * np.exp(-1j*phi)

for entry in bloch_table:
    cG   = entry["coeff"]
    kG   = entry["kG"]  # (NG,2)
    kvec = entry["k"]   # (2,)
    E_tab= entry["eps"].real

    # 1) kinetic energy
    E_kin = np.sum(np.abs(cG)**2 * hbar2_over_2m * np.sum(kG**2, axis=1))

    # 2) potential energy via Fourier sum
    orig_Gs = kG - kvec      # recover plane-wave G-vectors
    E_pot   = 0.0
    for i, Gi in enumerate(orig_Gs):
        for q, Vq_val in Vq.items():
            target = Gi + np.array(q)
            # find matching j
            j_idxs = np.where(np.all(np.isclose(orig_Gs, target, atol=1e-6), axis=1))[0]
            if j_idxs.size:
                j = j_idxs[0]
                E_pot += np.conj(cG[i]) * Vq_val * cG[j]
    E_pot = E_pot.real

    E_op = E_kin + E_pot
    Δ    = (E_op - E_tab) * 1e3  # meV to µeV

    rows.append({
        "E_table (meV)":    E_tab,
        "E_operator (meV)": E_op,
        "Δ (µeV)":          Δ
    })

df = pd.DataFrame(rows)
df

g list [array([-0.45169951,  0.78236649]), array([-0.45169951, -0.78236649]), array([ 9.03399011e-01, -2.21268941e-16])]


,E_table (meV),E_operator (meV),Δ (µeV)
0,-11.988679,-11.988679,-3.197442e-10
1,-4.706784,-4.706784,-9.237056e-11
2,-4.706784,-4.706784,1.927347e-10
3,-4.705304,-4.705304,9.947598e-11
4,2.262535,2.262535,1.598721e-11
5,2.262535,2.262535,5.817569e-11


In [50]:
df.style.format({
    'E_table (meV)'    : '{:.15f}',
    'E_operator (meV)' : '{:.15f}',
    'Δ (µeV)'          : '{:.6e}'
})


,E_table (meV),E_operator (meV),Δ (µeV)
0,-11.988679262840940,-11.988679262841259,-3.197442e-10
1,-4.706784167517021,-4.706784167517114,-9.237056e-11
2,-4.706784167517317,-4.706784167517124,1.927347e-10
3,-4.705303637306262,-4.705303637306162,9.947598e-11
4,2.262534788648869,2.262534788648885,1.598721e-11
5,2.262534788648813,2.262534788648871,5.817569e-11


#### Run VMC with Bloch determinant: with Bloch eigenfunctions (recall, from dispersion relation), and we choose correct k vectors

In [ ]:
with open("bloch_coeffs_3electrons.pkl", "rb") as fh:
    bloch = pickle.load(fh)          # list of 6 dictionaries
rows = []
for idx, entry in enumerate(bloch):
    kx, ky   = entry["k"]            # Bloch momentum components (nm⁻¹)
    eps      = entry["eps"]          # lowest-band eigenvalue (meV)
    n_G      = len(entry["coeff"])   # number of plane waves (should be 25)
    rows.append({"k_x (nm⁻¹)": kx,
                 "k_y (nm⁻¹)": ky,
                 "ε₀  (meV)"  : eps,
                 "# G-vectors": n_G})
df = pd.DataFrame(rows)_
df

,k_x (nm⁻¹),k_y (nm⁻¹),ε₀ (meV),# G-vectors
0,-0.000000,0.000000,-11.988679,25
1,-0.150567,-0.260789,-4.706784,25
2,-0.150567,0.260789,-4.706784,25


In [70]:
idx   = 0                     # pick the Γ-point
c_G   = bloch[idx]["coeff"]   # complex length-25 vector
kplus = bloch[idx]["kG"]      # 25 corresponding k+G vectors
sum = df["ε₀  (meV)"].sum()
print("energy sum", sum)
print('energy average', sum/6 )
print("\ncoeffs:", c_G)  # should be (25,)
print("\nnormalisation ⟨ψ|ψ⟩:", np.sum(np.abs(c_G)**2))  # should be 1.0

energy sum -21.402247597875277
energy average -3.5670412663125464

coeffs: [ 4.88042488e-03-0.00000000e+00j  5.94635876e-03+1.20259609e-02j
 -2.97522097e-03+3.88907796e-03j -7.73873874e-05+4.13746874e-04j
  5.14057733e-06+1.03947763e-05j  5.94635876e-03+1.20259609e-02j
  1.08567479e-01+7.26228402e-02j -8.18899182e-03+1.30370433e-01j
  5.96701190e-03+1.20659121e-02j  3.75782461e-04+1.89638671e-04j
 -2.97522097e-03+3.88907796e-03j -8.18899182e-03+1.30370433e-01j
  4.19699392e-01+8.48675359e-01j  1.08577457e-01+7.26253747e-02j
  4.89661645e-03-3.81859080e-06j -7.73873874e-05+4.13746874e-04j
  5.96701190e-03+1.20659121e-02j  1.08577457e-01+7.26253747e-02j
 -8.18494962e-03+1.30360965e-01j  5.94779841e-03+1.20252489e-02j
  5.14057733e-06+1.03947763e-05j  3.75782461e-04+1.89638671e-04j
  4.89661645e-03-3.81859080e-06j  5.94779841e-03+1.20252489e-02j
 -2.96235821e-03+3.87852818e-03j]

normalisation ⟨ψ|ψ⟩: 0.9999999999999969


In [ ]:
# put into mathematica npz format
bloch = pickle.load(open("bloch_coeffs_3electrons.pkl", "rb"))
k      = np.stack([b["k"]     for b in bloch])          # shape (N_e, 2)
eps    = np.stack([b["eps"]   for b in bloch])          # (N_e,)
coeffR = np.stack([b["coeff"].real for b in bloch])     # (N_e, N_G)
coeffI = np.stack([b["coeff"].imag for b in bloch])     # (N_e, N_G)
kG     = np.stack([b["kG"]    for b in bloch])          # (N_e, N_G, 2)
np.savez("bloch_3e.npz",
         kvec    = k,
         eps     = eps,
         coeffRe = coeffR,
         coeffIm = coeffI,
         kGvec   = kG)
print("wrote   bloch_3e.npz   — ready for Mathematica")

wrote   bloch_3e.npz   — ready for Mathematica


In [ ]:
# print out for Mathematica
bloch = pickle.load(open("bloch_coeffs_3electrons.pkl","rb"))
def fmt_complex(z):
    # Mathematica wants  a + b I
    return f"{z.real:.16g}{'+' if z.imag>=0 else ''}{z.imag:.16g}*I"
print("\n(* --- paste everything from the next line into Mathematica --- *)\n")
# --- kList --------------------------------------------------------------
print("kList  = {")
for b in bloch:
    print("  {", ", ".join(f"{x:.16g}" for x in b['k']), "},")
print("};\n")
# --- cList --------------------------------------------------------------
print("cList = {")
for b in bloch:
    coeffs = ", ".join(fmt_complex(z) for z in b['coeff'])
    print("  {", coeffs, "},")
print("};\n")
# --- kGList -------------------------------------------------------------
print("kGList = {")
for b in bloch:
    rows = ", ".join("{"+", ".join(f'{x:.16g}' for x in pair)+"}" for pair in b['kG'])
    print("  {", rows, "},")
print("};\n")

In [71]:
def slater_det(bloch, R):
    """ 
    input:
        bloch : list of dicts; bloch table
        R     : ndarray float, shape (N_e, 2)
    output:
        M     : (N_e, N_e) complex ndarray, Slater matrix  M_{ij} = φ_j(r_i)
        psi   : complex scalar, Wave-function value  Ψ(R) = (1/√N!) det M """

    N_e = len(bloch)
    if R.shape != (N_e, 2):
        raise ValueError(f"R must have shape ({N_e},2). Got {R.shape}")

    # ---------- build the Slater matrix ----------
    M = np.empty((N_e, N_e), dtype=np.complex128)

    for i, r in enumerate(R): # loop over orbitals (columns)
        for j, entry in enumerate(bloch):
            cG  = entry["coeff"]           # complex
            kG  = entry["kG"]              # k+G
            M[i, j] = np.sum(cG * np.exp(1j * kG @ r))

    # ---------- determinant with normalisation ----------
    detM = np.linalg.det(M)
    psi  = detM / math.sqrt(math.factorial(N_e))
    
    return M, psi

def psi_fn(R):
    """Wrapper that only returns Ψ(R)."""
    _, psi = slater_det(bloch, R)
    return psi

In [72]:
# choose a random electronic configuration inside the 3×3 super-cell N_e = len(bloch)
np.random.seed(42)
R_demo = np.random.rand(len(bloch), 2) * L     # shape (N_e,2)
M_demo, psi_demo = slater_det(bloch, R_demo)

print("Slater matrix M :", M_demo)
print("\nM_demo.shape:", M_demo.shape)
print("\nΨ(R) = det(M)/√N! =", psi_demo)
print("|Ψ(R)| =", abs(psi_demo))

Slater matrix M : [[ 0.43828896+0.88626538j -0.75221586+0.11528688j -0.59827822+0.7654162j ]
 [ 0.26762715+0.54116963j -0.3657312 -0.49587653j  0.09488271-0.44920832j]
 [ 0.44989454+0.90973305j -0.62784248+0.75349575j -0.40645042-0.92772238j]]

M_demo.shape: (3, 3)

Ψ(R) = det(M)/√N! = (-0.12770876706399437-0.6570587674918532j)
|Ψ(R)| = 0.669354728916528


Now in order To find
$$
E_{\rm kin}^{\rm loc}(R)
= -\frac{\hbar^2}{2m}
\sum_{i=1}^{N_e}\frac{\nabla_i^2\Psi(R)}{\Psi(R)}.
$$
use the central finite–difference approximation (second–order accurate in the step size h):
$$
\frac{\partial^2\Psi}{\partial r_{i\alpha}^2}\Bigg|_{R}
\approx
\frac{\Psi(R + h\,\hat e_{i\alpha}) - 2\Psi(R) + \Psi(R - h\,\hat e_{i\alpha})}{h^2},
$$

In [6]:
# def local_kinetic_energy(bloch, R, h=1.0e-4): # relook at the details
#     """ For configuration R, computes Σ_i ∇_i²Ψ(R) / Ψ(R)
#     using central finite differences. """

#     N_e = len(bloch)
#     psi0 = psi_fn(R)               # Ψ(R) at original positions

#     total = 0.0 + 0.0j
#     for i in range(N_e):           # loop electrons
#         for alpha in (0, 1):       # alpha = 0 (x) or 1 (y)
#             Rp = R.copy(); Rp[i, alpha] += h
#             Rm = R.copy(); Rm[i, alpha] -= h

#             psi_p = psi_fn(Rp)
#             psi_m = psi_fn(Rm)

#             lap_slice = (psi_p - 2.0*psi0 + psi_m) / h**2   # ∂²_{iα} Ψ
#             total += lap_slice / psi0                       # divide by Ψ(R)
#     return - hbar2_over_2m * total

# # random configuration inside the 3×3 box
# np.random.seed(42)
# R_demo = np.random.rand(len(bloch), 2) * L
# print("R_demo =", R_demo)

# local_kinetic_energy_complex = local_kinetic_energy(bloch, R_demo)
# print("\n -h2_over2m * Σ_i ∇_i²Ψ/Ψ =", local_kinetic_energy_complex)
# print("Real part (Kinentic energy)    =", local_kinetic_energy_complex.real)     # real part: kinetic Energy

In [73]:
#  ϕ , ∇ϕ , ∇²ϕ   for a single Bloch orbital at one position r
def _phi_grad_lap(entry, r, *, verbose=False):
    """ input:
    entry   : one dictionary from Bloch table, keys: "kG"  (n_G,2)  and  "coeff" (n_G,)
    r       :  one electron position (2,)  (x,y) in nm verbose : bool – print internal values (default False)
    Returns:
    phi  : complex scalar                ϕ(r)
    grad : complex ndarray, shape (2,)   ∇ϕ(r)
    lap  : complex scalar                ∇²ϕ(r) """
    
    kG  = entry["kG"]        # (n_G,2)
    cG  = entry["coeff"]     # (n_G,)
    phase = np.exp(1j * kG @ r)          # e^{i(k+G)·r}
    
    # ∑ c_G e^{i(k+G)·r}  (complex scalar)
    phi   = np.sum(cG * phase) 

    # gradient  ∑ c_G i(k+G) e^{i(k+G)·r}
    grad  = (1j * (cG * phase)[:, None] * kG).sum(axis=0)  # (2,)

    # Laplacian −∑ c_G |k+G|² e^{i(k+G)·r}
    k2    = (kG ** 2).sum(axis=1)
    lap   = -np.sum(cG * phase * k2)

    # if verbose:
    #     print(f"r          = {r}")
    #     print("phi        = ", phi)
    #     print("grad       = ", grad)
    #     print("laplacian  = ", lap)

    return phi, grad, lap

#  Local kinetic energy  (analytic derivatives)
def local_kinetic_energy_analytic(bloch, R, *, hbar2_over_2m=108.857,verbose=False):
    """ Returns T_loc(R) in meV  (real scalar) """
    N = len(bloch)

    # build Slater matrix and its derivatives
    M     = np.empty((N, N), dtype=np.complex128)
    gradφ = np.empty((N, N, 2), dtype=np.complex128)
    lapφ  = np.empty((N, N),     dtype=np.complex128)

    for i, r in enumerate(R):
        for j, entry in enumerate(bloch):
            φ, g, Δ = _phi_grad_lap(entry, r)  # set verbose=True here if needed
            M[i, j]      = φ
            gradφ[i, j]  = g
            lapφ[i, j]   = Δ

    Minv = np.linalg.inv(M)
    Tsum = 0.0

    for i in range(N):
        gln   = (Minv[:, i, None] * gradφ[i]).sum(axis=0)     # ∇_i ln Ψ  (2,)
        lapln = (Minv[:, i]       * lapφ[i]).sum()            # ∇²_i ln Ψ  (scalar)

        # grad2 = (gln.conj() * gln).sum()                      # |∇ ln Ψ|²
        grad2 = np.dot(gln, gln)         # no .conj()

        # if verbose:
        #     print("electron", i)
        #     print("∇lnΨ",  gln)
        #     print("∇²lnΨ", lapln)
        #     print("|∇lnΨ|²", grad2)

        #     # print("grad2 = (gln.conj() * gln).sum()", grad2)
        #     # print("np.dot(gln, gln)", grad2_new)
        
        Tsum += (lapln + grad2).real

    T_loc = -hbar2_over_2m * Tsum        # meV

    # if verbose:
    #     print("=== kinetic debug summary ===")
    #     print(f"Σ_i (∇²lnΨ+|∇lnΨ|²) = {Tsum:+.6e}")
    #     print(f"T_loc (meV)        = {T_loc:+.6f}")

    return T_loc

# test
np.random.seed(42)
R_demo = np.random.rand(len(bloch), 2) * L     # shape (N_e,2)
print("R_demo =", R_demo)
Ekin = local_kinetic_energy_analytic(bloch, R_demo, hbar2_over_2m=hbar2_over_2m)
print(Ekin)

R_demo = [[ 9.02379508 22.90555978]
 [17.63593004 14.42347886]
 [ 3.7589571   3.75837598]]
-48.88277504358307


In [95]:
# test the function _phi_grad_lap
entry = bloch[0] # Pick the first orbital (or any index you want)
r = np.array([0.5, 0.25])
phi, grad, lap = _phi_grad_lap(entry, r, verbose=True)

In [76]:
# test the local_kinetic_energy_analytic function
np.random.seed(42)
R = np.random.rand(len(bloch), 2) * L     # shape (N_e,2)
Ekin = local_kinetic_energy_analytic(bloch, R, hbar2_over_2m=hbar2_over_2m, verbose=True)
Ekin

-48.88277504358307

In [78]:
# if we use the BlochSlaterWF class, we get the same results as above functions:

from wavefunction.wf_bloch_slater import BlochSlaterWF
with open("bloch_coeffs_3electrons.pkl", "rb") as fh:
    bloch_table = pickle.load(fh)

# test
np.random.seed(42)
R = np.random.rand(len(bloch), 2) * L     # shape (N_e,2)
print("R =", R)

# --- Instantiate the wavefunction object ---
wf = BlochSlaterWF(bloch_table, L)
grad = wf.grad_log_psi(R)        # Shape (N_e, 2), complex
lap  = wf.laplacian_log_psi(R)   # Complex scalar

# --- Compute local kinetic energy ---
grad2 = np.sum(grad[:, 0] * grad[:, 0]    # v_x · v_x
               + grad[:, 1] * grad[:, 1]) # v_y · v_y
# old: grad2 = np.sum(np.abs(grad)**2)
kinetic = -hbar2_over_2m * (lap + grad2).real

print("Kinetic energy (meV):", kinetic)

R = [[ 9.02379508 22.90555978]
 [17.63593004 14.42347886]
 [ 3.7589571   3.75837598]]
Kinetic energy (meV): -48.882775043583074


Run the Monte Carlo (no variational or optimization)

In [79]:
#  MONTE-CARLO ENGINE   (Metropolis-Hastings for |Ψ|²)
class MetropolisVMC:
    """ Very lightweight Variational Monte-Carlo driver that relies ONLY on
      • psi_fn(R)                    – complex wavefunction value
      • local_kinetic_energy(bloch,R)
      • energy_static(R)  (can be a dummy lambda R: 0.0)
    """

    def __init__(self, *,
                 bloch,
                 box_length,
                 hbar2_over_2m,
                 energy_static_fn,
                 step_size=0.25):
        
        self.bloch      = bloch
        self.L          = box_length               # nm
        self.N_e        = len(bloch)               # number of electrons
        self.h2m        = hbar2_over_2m            # meV·nm²
        self.step_size  = step_size                # proposal displacement (nm)
        self.E0         = energy_static_fn

    # ── kinetic + external + Coulomb … full local energy at R ──────────
    def local_energy(self, R):
        Ekin = local_kinetic_energy_analytic(self.bloch, R)          # meV                         # Local KE function
        # print("local energy", Ekin.real + self.E0(R))
        return Ekin.real + self.E0(R)                       # real scalar

    # ── one Metropolis sweep over all electrons ────────────────────────
    def _metropolis(self, R):
        logp0 = 2.0 * np.log(np.abs(psi_fn(R)))          # ln |Ψ|²
        for i in range(self.N_e):
            trial = R.copy()
            # propose tiny random move inside the simulation cell
            trial[i] = (R[i] + np.random.uniform(-self.step_size,
                                                 +self.step_size,
                                                 2)) % self.L
            logp1 = 2.0 * np.log(np.abs(psi_fn(trial)))
            # accept with probability min(1, e^{logp1-logp0})
            if np.log(np.random.rand()) < (logp1 - logp0):
                R[i]  = trial[i]
                logp0 = logp1
        return R

    # ── generate Ncfg configurations after burn-in ─────────────────────
    def sample_configs(self, Ncfg=400, burn=1000):
        R = np.random.rand(self.N_e, 2) * self.L
        cfgs = []
        for step in range(Ncfg + burn):
            R = self._metropolis(R)
            if step >= burn:
                cfgs.append(R.copy())
        return np.array(cfgs)

    # ── main routine: accumulate energy blocks and statistical error ──
    def run(self,
            n_blocks          = 20,
            cfgs_per_block    = 400,
            burn_in           = 2000,
            verbose           = True):
        avg_block = []
        for blk in range(n_blocks):
            # collect block of configurations
            cfgs = self.sample_configs(cfgs_per_block, burn=burn_in if blk == 0 else 0)
            # local energies of that block
            Eloc = np.array([self.local_energy(R) for R in cfgs])
            avg_block.append(Eloc.mean())
            if verbose:
                print(f"block {blk+1:02d}/{n_blocks}  "
                      f"<E_loc> = {Eloc.mean(): .6f} meV   "
                      f"σ = {Eloc.std(ddof=1):.4f}")
        avg_block = np.array(avg_block)
        mean = avg_block.mean()
        err  = avg_block.std(ddof=1)/np.sqrt(n_blocks)
        return mean, err

In [80]:
# analytic function for local KE, 60 blocks, used moire potential part of satis energy
vmc = MetropolisVMC(
        bloch = bloch, box_length = L, hbar2_over_2m = hbar2_over_2m,
        # energy_static_fn  = energy_static,
        energy_static_fn  = moire_potential, #change it back later, now we wanna only include moire potential but not the Vee
        step_size = 0.25 # nm
      )

E_mean, E_err = vmc.run(n_blocks=80,
                        cfgs_per_block=400,
                        burn_in=2000,
                        verbose=True)

print("\n──── VMC result ───────────────────────────────────────")
print("E_mean", E_mean)
# print(f" E / N_e  = {E_mean / len(bloch): .6f}  ± {E_err / len(bloch): .6f}  meV")


block 01/80  <E_loc> =  95.731195 meV   σ = 55.9519
block 02/80  <E_loc> = -21.989699 meV   σ = 46.3330
block 03/80  <E_loc> = -32.010766 meV   σ = 80.5184
block 04/80  <E_loc> = -23.371263 meV   σ = 46.2693
block 05/80  <E_loc> = -15.180898 meV   σ = 39.1440
block 06/80  <E_loc> =  3.580199 meV   σ = 50.9890
block 07/80  <E_loc> = -70.689939 meV   σ = 91.2286
block 08/80  <E_loc> = -21.572661 meV   σ = 39.8094
block 09/80  <E_loc> =  36.716541 meV   σ = 76.8880
block 10/80  <E_loc> = -7.986602 meV   σ = 97.7780
block 11/80  <E_loc> =  83.796265 meV   σ = 85.7691
block 12/80  <E_loc> = -2.528580 meV   σ = 45.1391
block 13/80  <E_loc> = -20.841033 meV   σ = 125.8985
block 14/80  <E_loc> = -35.495497 meV   σ = 64.6729
block 15/80  <E_loc> = -47.458499 meV   σ = 17.1914
block 16/80  <E_loc> = -82.574745 meV   σ = 76.8757
block 17/80  <E_loc> = -141.183667 meV   σ = 70.8535
block 18/80  <E_loc> = -6.931936 meV   σ = 65.9220
block 19/80  <E_loc> = -21.136788 meV   σ = 48.4808
block 20/80  <

In [90]:
# grab 3 random configurations
for _ in range(3):
    R = np.random.rand(len(bloch), 2) * L
    print("Eloc =", vmc.local_energy(R))


Eloc = -217.23734429008675
Eloc = -15.773472920352297
Eloc = -28.940627293786562


In [ ]:
# analytic function for local KE, 60 blocks, used full energy_static
vmc = MetropolisVMC(
        bloch = bloch, box_length = L, hbar2_over_2m = hbar2_over_2m,
        energy_static_fn  = energy_static,
        # energy_static_fn  = moire_potential, #change it back later, now we wanna only include moire potential but not the Vee
        step_size = 0.25 # nm
      )

E_mean, E_err = vmc.run(n_blocks=80,
                        cfgs_per_block=400,
                        burn_in=2000,
                        verbose=True)

print("\n──── VMC result ───────────────────────────────────────")
print("E_mean", E_mean)
# print(f" E / N_e  = {E_mean / len(bloch): .6f}  ± {E_err / len(bloch): .6f}  meV")


block 01/80  <E_loc> = -111.488182 meV   σ = 80.2903
block 02/80  <E_loc> = -68.831914 meV   σ = 82.7680
block 03/80  <E_loc> = -36.700551 meV   σ = 120.4222
block 04/80  <E_loc> =  50.716753 meV   σ = 96.9755
block 05/80  <E_loc> = -9.841972 meV   σ = 358.9037
block 06/80  <E_loc> = -112.529869 meV   σ = 54.0379
block 07/80  <E_loc> = -252.299998 meV   σ = 1585.4816
block 08/80  <E_loc> =  16.225678 meV   σ = 243.2173
block 09/80  <E_loc> = -159.156973 meV   σ = 289.4021
block 10/80  <E_loc> = -143.775271 meV   σ = 140.7263
block 11/80  <E_loc> =  68.038532 meV   σ = 88.8567
block 12/80  <E_loc> = -333.239119 meV   σ = 553.2971
block 13/80  <E_loc> = -145.601372 meV   σ = 125.5895
block 14/80  <E_loc> = -29.618553 meV   σ = 110.0223
block 15/80  <E_loc> = -159.280872 meV   σ = 178.5713
block 16/80  <E_loc> = -8.119245 meV   σ = 101.1762
block 17/80  <E_loc> =  13.085255 meV   σ = 891.8506
block 18/80  <E_loc> = -81.997229 meV   σ = 171.3749
block 19/80  <E_loc> = -162.071313 meV   σ =

In [ ]:
# # non-analytic function for local KE, 60 blocks, used full energy_static
# vmc = MetropolisVMC(
#         bloch             = bloch,
#         box_length        = L,                 # nm
#         hbar2_over_2m     = hbar2_over_2m,     # meV·nm²
#         energy_static_fn  = energy_static,
#         # energy_static_fn  = moire_potential, #change it back later, now we wanna only include moire potential but not the Vee
#         step_size         = 0.25               # nm
#       )

# E_mean, E_err = vmc.run(n_blocks=60,
#                         cfgs_per_block=400,
#                         burn_in=2000,
#                         verbose=True)

# print("\n──── VMC result ───────────────────────────────────────")
# # the results is showing the energy per electron
# print(f" E / N_e  = {E_mean / len(bloch): .6f}  ± {E_err / len(bloch): .6f}  meV")


block 01/60  <E_loc> = -37.390610 meV   σ = 47.0068
block 02/60  <E_loc> =  166.010815 meV   σ = 101.2180
block 03/60  <E_loc> =  60.756018 meV   σ = 134.6548
block 04/60  <E_loc> =  71.203557 meV   σ = 118.0199
block 05/60  <E_loc> =  58.057724 meV   σ = 94.1204
block 06/60  <E_loc> =  55.218430 meV   σ = 45.2249
block 07/60  <E_loc> = -6.605122 meV   σ = 104.6434
block 08/60  <E_loc> = -24.369857 meV   σ = 87.2232
block 09/60  <E_loc> =  187.837737 meV   σ = 66.6031
block 10/60  <E_loc> =  101.198677 meV   σ = 60.2409
block 11/60  <E_loc> =  112.121551 meV   σ = 67.2140
block 12/60  <E_loc> =  42.603960 meV   σ = 76.9433
block 13/60  <E_loc> =  149.584527 meV   σ = 68.0856
block 14/60  <E_loc> =  39.525611 meV   σ = 82.3082
block 15/60  <E_loc> =  129.489166 meV   σ = 74.5278
block 16/60  <E_loc> =  44.875636 meV   σ = 74.3094
block 17/60  <E_loc> =  38.685904 meV   σ = 73.3087
block 18/60  <E_loc> = -8.256043 meV   σ = 46.0110
block 19/60  <E_loc> =  94.907748 meV   σ = 74.1877
bloc

In [14]:
# np.random.seed(42)
R = np.random.rand(len(bloch), 2) * L
# local KE:
Ekin = local_kinetic_energy_analytic(bloch, R).real
# energy static (R):
Vext = np.sum(moire_potential(R))
Vee  = coulomb_ewald_2D(R)
print(Ekin, Vext, Vee, "  total =", Ekin+Vext+Vee)


-216.55923969932212 -44.82245080075108 6.536407813938048   total = -254.84528268613516


Test through VMC

In [ ]:
import pickle, numpy as np, time
from wavefunction.wf_bloch_slater import BlochSlaterWF
from moire_model      import energy_static, L
from vmc_core         import VMC
# 1) wavefunction
bloch = pickle.load(open("bloch_coeffs.pkl","rb"))
wf    = BlochSlaterWF(bloch, L)

# 2) VMC engine
vmc   = VMC(wf, energy_static, hbar2_over_2m=108.857, box_length=L)

# 3) production sampling: 40 blocks × 400 configs
E_blocks = []
t0 = time.time()
for _ in range(80):
    E_mean, _, _ = vmc.step(n_cfg=400, lr=0.0,   # lr=0 → no optimisation
                            clip=100.0)          # clip ignored here
    print(f"block {len(E_blocks)+1:02d}  <E_loc> = {E_mean:.6f} meV")
    E_blocks.append(E_mean)

E_blocks = np.array(E_blocks)
E_mean   = E_blocks.mean()
E_err    = E_blocks.std(ddof=1)/np.sqrt(len(E_blocks))

print("\n---  3×3 super-cell  |  spin-polarised  |  single Slater  ---")
print(f"E/N  = {E_mean/wf.N_e: .6f}  ± {E_err/wf.N_e: .6f}  meV")
print(f"elapsed {time.time()-t0:.1f} s,   {len(E_blocks)*400} configurations")


block 01  <E_loc> = -89.783297 meV
block 02  <E_loc> = -58.343168 meV
block 03  <E_loc> = -92.568130 meV
block 04  <E_loc> = -108.907901 meV
block 05  <E_loc> = -66.658347 meV
block 06  <E_loc> = -14.205713 meV
block 07  <E_loc> = -89.364963 meV
block 08  <E_loc> = -27.307555 meV
block 09  <E_loc> = -68.997604 meV
block 10  <E_loc> = -158.302230 meV
block 11  <E_loc> = -40.613445 meV
block 12  <E_loc> = -93.238165 meV
block 13  <E_loc> = -150.424896 meV
block 14  <E_loc> = -121.512450 meV
block 15  <E_loc> = -58.058565 meV
block 16  <E_loc> = -139.574184 meV
block 17  <E_loc> = -126.864454 meV
block 18  <E_loc> = -177.642733 meV
block 19  <E_loc> = -151.631370 meV
block 20  <E_loc> = -341.333487 meV
block 21  <E_loc> = -186.686638 meV
block 22  <E_loc> = -32.484683 meV
block 23  <E_loc> = -159.534851 meV
block 24  <E_loc> = -141.077250 meV
block 25  <E_loc> = -118.426454 meV
block 26  <E_loc> = -32.363585 meV
block 27  <E_loc> = -112.870847 meV
block 28  <E_loc> = -131.770522 meV
block

#### Run SlaterNet (old)

In [15]:
import time, torch
from vmc_core      import VMC            # your existing file
from torch import nn
N_e = 6
box_length = a_m * 3.0

In [16]:
class FeedForwardLayer(nn.Module):
    """ A single feed-forward layer with a tanh activation function.
        The input is added to the output of the layer. """
    
    def __init__(self, L: int) -> None: # L: layer of width d_L
        super().__init__()
        self.Wl_1p = nn.Linear(L, L)   # W^(l+1) h^l + b^(l+1)
        self.tanh = nn.Tanh()       # (nonlinear) hyperbolic tangent activation function

    def forward(self, hl: torch.Tensor) -> torch.Tensor:
        return hl + self.tanh(self.Wl_1p(hl))  # input should be of shape (N, L): h^l + tanh( W^(l+1) h^l + b^(l+1) )

class SlaterNet(nn.Module):
    # def __init__(self, a: float, N: int, L: int = 4, num_layers: int = 3) -> None: # a: lattice constant
    def __init__(self, a, N, L = 64, num_layers = 3) -> None:  # L: layer of width d_L; a: lattice constant

        super().__init__()
        self.N = N
        self.L = L
        self.a = a
        self.num_layers = num_layers
        
        G_vectors = torch.from_numpy(np.array(b_vectors(a))).float()
        self.G1_T = G_vectors[0].unsqueeze(-1)
        self.G2_T = G_vectors[1].unsqueeze(-1)

        # input embedding matrix: projects 4 features to L-dim
        self.W_0 = nn.Linear(4, L, bias=False)
        self.MLP_layers = nn.ModuleList( # MLP layers
            [FeedForwardLayer(L) for _ in range(num_layers)]
        )

        # matrix to hold the projection vectors (complex projectors for orbital) 
        # w_2j and w_2j+1 (one for real one for complex) for j = 0, ... N-1 (6 electrons)
        self.complex_proj = nn.Parameter(
            torch.complex(real=torch.randn(L, N), imag=torch.randn(L, N))
        )
        # self.denominator = math.sqrt(math.factorial(N))

    def forward(self, R: torch.Tensor) -> torch.Tensor:  # R should be of shape (N, 2)
        
        G1_R = torch.matmul(R, self.G1_T)         # compute the periodic features
        G2_R = torch.matmul(R, self.G2_T)
        features_R = torch.cat(
            (torch.sin(G1_R), torch.sin(G2_R), torch.cos(G1_R), torch.cos(G2_R)), dim=1
        ) # shape (N, 4)

        # embed in higher_dimensional space to get h^0
        h = self.W_0(features_R)

        # pass through MLP layers
        for layer in self.MLP_layers:
            h = layer(h)

        # slater matxix
        WF_matrix = torch.matmul(h.to(torch.complex64), self.complex_proj)
        determinant = torch.linalg.det(WF_matrix)
        # result = determinant / self.denominator
        return determinant

In [17]:
# test the NN
net = SlaterNet(a=a_m, N=6, L=64, num_layers=3).to("cpu")
# net.eval()
R = torch.rand(6,2) * a_m   # a random electron config
psi_val = net(R)            # complex wave-function amplitude
print("psi_value", psi_val)
prob    = psi_val.abs()**2  # probability density  |Ψ|²
print("prob", prob)

psi_value tensor(2219.9314-3641.9214j, grad_fn=<LinalgDetBackward0>)
prob tensor(18191686., grad_fn=<PowBackward0>)


In [18]:
# test the SlaterNet physics
def random_config(N, box_length):
    """Uniformly random electron positions in the simulation cell."""
    return torch.rand(N, 2) * box_length          # (N,2) tensor

def antisymmetry_test(net, box_length):
    R  = random_config(net.N, box_length)
    ψ1 = net(R)

    # swap electron 0 and 1
    R_swapped = R.clone()
    R_swapped[[0, 1]] = R_swapped[[1, 0]]
    ψ2 = net(R_swapped)

    print("Antisymmetry test:")
    print("  ψ(R)          =", ψ1)
    print("  ψ(R_swap)     =", ψ2)
    print("  ratio ψ2/ψ1   =", ψ2 / ψ1)
    print("  Expected      ≈ -1\n")

def periodicity_test(net, box_length, direct_a1=None):
    """
    Adds one direct lattice vector a1 to electron 0. If you use a square lattice, a1 = (a_m, 0).
    """
    # guess a1 if not supplied: square box of length `box_length`
    if direct_a1 is None:
        direct_a1 = torch.tensor([box_length, 0.0])

    R  = random_config(net.N, box_length)
    ψ1 = net(R)

    # translate electron 0 by a lattice vector (mod the box)
    R_trans = R.clone()
    R_trans[0] = (R_trans[0] + direct_a1) % box_length
    ψ2 = net(R_trans)

    print("Periodicity test:")
    print("  ψ(R)                    =", ψ1)
    print("  ψ(R + a1 on electron 0) =", ψ2)
    print("  Expected                ≈ 0\n")

antisymmetry_test(net, box_length=a_m) # swap two electrons in R; psi_val must flip sign.
periodicity_test(net, box_length=a_m, direct_a1=torch.tensor([a_m, 0.0])) # add a lattice vector to any coord; psi_val stay same.


Antisymmetry test:
  ψ(R)          = tensor(3665.9053-29159.j, grad_fn=<LinalgDetBackward0>)
  ψ(R_swap)     = tensor(-3665.9053+29159.j, grad_fn=<LinalgDetBackward0>)
  ratio ψ2/ψ1   = tensor(-1.+0.j, grad_fn=<DivBackward0>)
  Expected      ≈ -1

Periodicity test:
  ψ(R)                    = tensor(-12554.4072+2992.8213j, grad_fn=<LinalgDetBackward0>)
  ψ(R + a1 on electron 0) = tensor(-12554.4072+2992.8213j, grad_fn=<LinalgDetBackward0>)
  Expected                ≈ 0



$$
T_\text{loc}(\mathbf R)\;=\;-\frac{\hbar^{2}}{2m}\sum_{i=1}^{N}
\bigl[\;\nabla_i^{2}\ln\!|\Psi(\mathbf R)|\;+\;|\nabla_i\ln\!\Psi(\mathbf R)|^{2}\bigr].
\tag{24 in the paper}
$$

$$
\nabla_i\ln\Psi
      \;=\;\sum_{j=1}^{N}\,(\Phi^{-1})_{j\,i}\,\nabla_i \Phi_{i\,j}.
\tag{25}
$$

question: is it dot product or conjugate?

In [19]:
def local_kinetic_energy_slaternet(slater_net, R_np, hbar2_over_2m=hbar2_over_2m, device="cpu"): #R_np positions of electrons
        
    N = R_np.shape[0] # number of electrons
    R = torch.tensor(R_np, dtype=torch.float32, device=device, requires_grad=True) # (N,2)
    
    # Build the Slater matrix (copied from the forward pass of SlaterNet)
    G1_R = torch.matmul(R, slater_net.G1_T)
    G2_R = torch.matmul(R, slater_net.G2_T)
    features_R = torch.cat((torch.sin(G1_R), torch.sin(G2_R), torch.cos(G1_R), torch.cos(G2_R)), dim=1)
    h = slater_net.W_0(features_R)
    for layer in slater_net.MLP_layers:
        h = layer(h)
    WF_matrix = torch.matmul(h.to(torch.complex64), slater_net.complex_proj)  # (N,N)
    
    # Ψ(R) and Φ^{-1}
    det_wf = torch.linalg.det(WF_matrix)
    Minv = torch.linalg.inv(WF_matrix)
    
    # Loop over electrons, building ∇ln Ψ and ∇²ln Ψ
    Tsum = 0.0
    for i in range(N): # electron i
        # Take gradients w.r.t. r_i (the ith row of R); PyTorch trick: set requires_grad=True for *all* of R, then use autograd.grad for each i's coordinates.
        phi_i = WF_matrix[i]  # row i → ϕ_j(r_i); shape (N,)
        grads_phi = []
        laps_phi = []

        for j in range(N): # loop over orbitals (columns)
            # autograd.grad differentiates φᵢⱼ with respect to all coordinates in R, then we pick the slice [i] that belongs to electron i.
            grad_phi_j = torch.autograd.grad(
                phi_i[j], R, grad_outputs=torch.ones_like(phi_i[j]), create_graph=True, retain_graph=True
            )[0][i]  # (2,)
            grads_phi.append(grad_phi_j) #store every orbital gradient in grads_phi[j]
            lap_phi_j = 0.0

            # Laplacian ∇²φᵢⱼ = ∂²/∂x² φᵢⱼ + ∂²/∂y² φᵢⱼ
            # We use autograd.grad to compute the second derivative of φᵢⱼ with respect to R.
            for d in range(2):
                grad2 = torch.autograd.grad(
                    grad_phi_j[d], R, grad_outputs=torch.ones_like(grad_phi_j[d]), retain_graph=True
                )[0][i][d]
                lap_phi_j += grad2
            laps_phi.append(lap_phi_j)
        
        # Assemble ∇lnΨ and ∇²lnΨ with Jacobi’s formula
        grads_phi = torch.stack(grads_phi)
        laps_phi = torch.stack(laps_phi)
        Minv_col = Minv[:, i]
        gln = torch.sum(Minv_col[:, None] * grads_phi, dim=0)
        lapln = torch.sum(Minv_col * laps_phi)
        grad2 = torch.dot(gln, gln) # |∇lnΨ|² need conjugate?
        Tsum += (lapln + grad2).real
    T_loc = -hbar2_over_2m * Tsum
    return T_loc

In [20]:
net

SlaterNet(
  (W_0): Linear(in_features=4, out_features=64, bias=False)
  (MLP_layers): ModuleList(
    (0): FeedForwardLayer(
      (Wl_1p): Linear(in_features=64, out_features=64, bias=True)
      (tanh): Tanh()
    )
    (1): FeedForwardLayer(
      (Wl_1p): Linear(in_features=64, out_features=64, bias=True)
      (tanh): Tanh()
    )
    (2): FeedForwardLayer(
      (Wl_1p): Linear(in_features=64, out_features=64, bias=True)
      (tanh): Tanh()
    )
  )
)

In [21]:
# test the KE
R_np = np.random.rand(N_e, 2) * box_length
T_loc = local_kinetic_energy_slaternet(net, R_np, hbar2_over_2m=hbar2_over_2m, device="cpu")
print("Local kinetic energy (PyTorch tensor):", T_loc)
print("As float (for sanity):", float(T_loc))

def test_kinetic_energy_function(slater_net, hbar2_over_2m, box_length):
    N = slater_net.N
    R_np = np.random.rand(N, 2) * box_length
    T_loc = local_kinetic_energy_slaternet(slater_net, R_np, hbar2_over_2m, device="cpu")
    print("Kinetic energy (tensor):", T_loc)
    print("Kinetic energy (float):", float(T_loc))
    print("Is tensor?", isinstance(T_loc, torch.Tensor))
    # Swap two electrons and check
    R_swap = R_np.copy()
    R_swap[[0,1]] = R_swap[[1,0]]
    T_swap = local_kinetic_energy_slaternet(slater_net, R_swap, hbar2_over_2m, device="cpu")
    print("After swapping e0 and e1, kinetic energy:", float(T_swap))
    print("Difference (should be small for random net):", float(T_loc)-float(T_swap))

test_kinetic_energy_function(net, hbar2_over_2m, box_length)


Local kinetic energy (PyTorch tensor): tensor(142.0231, grad_fn=<MulBackward0>)
As float (for sanity): 142.0230712890625
Kinetic energy (tensor): tensor(484.9024, grad_fn=<MulBackward0>)
Kinetic energy (float): 484.90240478515625
Is tensor? True
After swapping e0 and e1, kinetic energy: 484.90240478515625
Difference (should be small for random net): 0.0


In [22]:
#  same function, again: MONTE-CARLO ENGINE but replaced with local_kinetic_energy_slaternet   (Metropolis-Hastings for |Ψ|²)
class MetropolisVMC:
    """ Variational Monte-Carlo driver that relies ONLY on
      • psi_fn(R)                    – complex wavefunction value
      • local_kinetic_energy(bloch,R)
      • energy_static(R)  (can be a dummy lambda R: 0.0)
    """

    def __init__(self, *,
                bloch,
                box_length,
                hbar2_over_2m,
                energy_static_fn,
                step_size=0.25,
                local_kinetic_energy=None):  # <-- add this
        
        self.bloch      = bloch
        self.L          = box_length
        self.N_e        = len(bloch)
        self.h2m        = hbar2_over_2m
        self.step_size  = step_size
        self.E0         = energy_static_fn   # here we use the energy_static function from moire_model
        self.local_kinetic_energy = local_kinetic_energy  # function to compute local kinetic energy

    # ── kinetic + external + Coulomb … full local energy at R ──────────
    def local_energy(self, R):
        Ekin = self.local_kinetic_energy(R)          # meV  # Local KE function
        # print("local energy", Ekin.real + self.E0(R))
        return Ekin.real + self.E0(R)  

    # ── one Metropolis sweep over all electrons ────────────────────────
    def _metropolis(self, R): 
        """ Perform one Metropolis step on the configuration R."""
        # logp0 = 2 * log|Ψ(R)|      # ln P(R)
        # logp1 = 2 * log|Ψ(R')|     # ln P(R')
        # ratio = exp(logp1 - logp0) = P(R') / P(R)
        logp0 = 2.0 * np.log(np.abs(psi_fn(R)))          # ln |Ψ|²
        for i in range(self.N_e):
            trial = R.copy()
            # propose tiny random move inside the simulation cell
            trial[i] = (R[i] + np.random.uniform(-self.step_size, +self.step_size, 2)) % self.L
            logp1 = 2.0 * np.log(np.abs(psi_fn(trial)))
            # accept with probability min(1, e^{logp1-logp0})
            if np.log(np.random.rand()) < (logp1 - logp0):
                R[i]  = trial[i]
                logp0 = logp1
        return R

    # ── generate Ncfg configurations after burn-in ─────────────────────
    def sample_configs(self, Ncfg=400, burn=1000):
        """ Generate Ncfg configurations after burn-in.""" 
        # “Burn-in” = ignoring the first part of the random walk until the walker is actually sampling the desired |Ψ|² distribution. 
        R = np.random.rand(self.N_e, 2) * self.L  # sampling inside the box
        cfgs = []
        for step in range(Ncfg + burn):
            R = self._metropolis(R)
            if step >= burn:
                cfgs.append(R.copy())
        return np.array(cfgs)

    # ── main routine: accumulate energy blocks and statistical error ──
    def run(self,
            n_blocks          = 20,
            cfgs_per_block    = 400,
            burn_in           = 2000,
            verbose           = True):
        """ Run the VMC simulation."""
        avg_block = []
        for blk in range(n_blocks):
            # collect block of configurations
            cfgs = self.sample_configs(cfgs_per_block, burn=burn_in if blk == 0 else 0)
            # local energies of that block
            Eloc = np.array([self.local_energy(R) for R in cfgs])
            avg_block.append(Eloc.mean())
            if verbose:
                print(f"block {blk+1:02d}/{n_blocks}  "
                      f"<E_loc> = {Eloc.mean(): .6f} meV   "
                      f"σ = {Eloc.std(ddof=1):.4f}")
        # average over blocks         
        avg_block = np.array(avg_block)
        mean = avg_block.mean()
        err  = avg_block.std(ddof=1)/np.sqrt(n_blocks)
        return mean, err
    # One configuration R: a single sample from P
    # One block: 400 successive configurations whose average energy we treat as one data point.
    # All blocks together → a small set of ≈ independent data points; their mean is best energy estimate, their scatter gives the statistical error.

In [23]:
# training:
slater_net = SlaterNet(a=a_m, N=6, L=64, num_layers=3)
slater_net.train()  # Put in training mode

optimizer = torch.optim.Adam(slater_net.parameters(), lr=1e-3)  # Start with Adam; you can try KFAC later

n_epochs = 100           # Number of optimization steps
samples_per_epoch = 128  # Number of VMC samples per optimization step

# ----- Metropolis sampler for configurations -----
def psi_fn(R):
    """ Computes the wavefunction value for a given configuration R."""
    R_tensor = torch.tensor(R, dtype=torch.float32)
    with torch.no_grad():
        val = slater_net(R_tensor)
    return val.item()
globals()['psi_fn'] = psi_fn  # For MetropolisVMC to use

# Use a "dummy" local_kinetic_energy for sampling, since we'll compute energy via gradients below
def dummy_kinetic_energy(R):
    return 0.0

sampler = MetropolisVMC(
    bloch=[0]*N_e,
    box_length=box_length,
    hbar2_over_2m=hbar2_over_2m,
    energy_static_fn=lambda R: 0.0,
    step_size=0.25,
    local_kinetic_energy=dummy_kinetic_energy  # Sampling does NOT use energy
)

# -------- Training loop --------
for epoch in range(n_epochs):
    # 1. Sample configurations from |Psi|^2
    configs = sampler.sample_configs(Ncfg=samples_per_epoch, burn=200)
    
    E_locals = []
    for R in configs:
        T = local_kinetic_energy_slaternet(slater_net, R, hbar2_over_2m=hbar2_over_2m, device="cpu")
        V = energy_static(R)
        V = torch.tensor(V, dtype=T.dtype)
        E_locals.append(T+V)
    loss = torch.stack(E_locals).mean()
    
    # 4. Backpropagate and update parameters
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # 5. Print progress
    print(f"Epoch {epoch+1:03d}/{n_epochs}  <E_loc> = {loss.item(): .6f} meV")


Epoch 001/100  <E_loc> =  484.143280 meV
Epoch 002/100  <E_loc> =  295.378174 meV
Epoch 003/100  <E_loc> =  281.443970 meV
Epoch 004/100  <E_loc> =  298.797028 meV
Epoch 005/100  <E_loc> =  270.323639 meV
Epoch 006/100  <E_loc> =  298.697815 meV
Epoch 007/100  <E_loc> =  324.693634 meV
Epoch 008/100  <E_loc> =  341.726715 meV
Epoch 009/100  <E_loc> =  355.890045 meV
Epoch 010/100  <E_loc> =  382.566650 meV
Epoch 011/100  <E_loc> =  423.279480 meV
Epoch 012/100  <E_loc> =  441.336792 meV
Epoch 013/100  <E_loc> =  375.381104 meV
Epoch 014/100  <E_loc> =  376.945221 meV
Epoch 015/100  <E_loc> =  249.848511 meV
Epoch 016/100  <E_loc> =  239.819519 meV
Epoch 017/100  <E_loc> =  483.598083 meV
Epoch 018/100  <E_loc> =  301.409210 meV
Epoch 019/100  <E_loc> =  156.583923 meV
Epoch 020/100  <E_loc> =  273.509399 meV
Epoch 021/100  <E_loc> =  512.831238 meV
Epoch 022/100  <E_loc> =  488.912170 meV
Epoch 023/100  <E_loc> =  406.002411 meV
Epoch 024/100  <E_loc> =  304.182343 meV
Epoch 025/100  <

#### others

In [22]:
import scipy.constants as const

# Physical constants
hbar = const.hbar      # Reduced Planck constant (J·s)
m_e = const.m_e        # Electron mass (kg)

# Compute ℏ²/(2m_e) in J·m²
value_j_m2 = hbar**2 / (2 * m_e)

# Convert to eV·m²
value_ev_m2 = value_j_m2 / const.e

# Convert to meV·nm²: 
# 1 eV = 1000 meV, 1 m² = (1e9 nm)² = 1e18 nm²
value_mev_nm2 = value_ev_m2 * 1e3 * 1e18

# Output results
print(f"ℏ²/(2 m_e) = {value_j_m2:.6e} J·m²")
print(f"         = {value_ev_m2:.6e} eV·m²")
print(f"         = {value_mev_nm2:.6f} meV·nm²")


ℏ²/(2 m_e) = 6.104264e-39 J·m²
         = 3.809982e-20 eV·m²
         = 38.099821 meV·nm²
